In [2]:
## Al active listing alpha vantage

import csv
import requests
import pandas as pd

API_KEY = os.environ['av_key']



# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = f'https://www.alphavantage.co/query?function=LISTING_STATUS&apikey={API_KEY}&entitlement=delayed'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    data = pd.DataFrame(columns=['symbol', 'name', 'exchange', 'assetType', 'ipoDate', 'delistingDate', 'status'], data=my_list[1:])
    print(data)

      symbol                                             name   exchange  \
0          A                         Agilent Technologies Inc       NYSE   
1         AA                                       Alcoa Corp       NYSE   
2        AAA  ALTERNATIVE ACCESS FIRST PRIORITY CLO BOND ETF   NYSE ARCA   
3       AAAU                  Goldman Sachs Physical Gold ETF       BATS   
4       AACG                            ATA Creativity Global     NASDAQ   
...      ...                                              ...        ...   
11721   ZYME                                 Zymeworks BC Inc     NASDAQ   
11722   ZYNE                      Zynerba Pharmaceuticals Inc     NASDAQ   
11723   ZYXI                                        Zynex Inc     NASDAQ   
11724    ZZK                                                   NYSE ARCA   
11725    ZZZ                              TEST TICKER FOR UTP  NYSE ARCA   

      assetType     ipoDate delistingDate  status  
0         Stock  1999-11-18        

In [2]:
import numpy as np

In [7]:
data.loc[data['assetType'] == 'Stock']

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,null,Active
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,null,Active
4,AACG,ATA Creativity Global,NASDAQ,Stock,2008-01-29,null,Active
5,AACT,Ares Acquisition Corporation II - Class A,NYSE,Stock,2023-06-12,null,Active
6,AACT-U,Ares Acquisition Corporation II - Units (1 Ord...,NYSE,Stock,2023-04-21,null,Active
...,...,...,...,...,...,...,...
11688,ZYME,Zymeworks BC Inc,NASDAQ,Stock,2017-04-28,null,Active
11689,ZYNE,Zynerba Pharmaceuticals Inc,NASDAQ,Stock,2015-08-05,null,Active
11690,ZYXI,Zynex Inc,NASDAQ,Stock,2012-07-23,null,Active
11691,ZZK,,NYSE ARCA,Stock,2020-07-22,null,Active


# Downloading timer

In [41]:
data_tickers.loc[(data_tickers['assetType'] == 'Stock') & (data_tickers['symbol'] >= 'BC/PA'), 'symbol']

1053     BC/PA
1054     BC/PB
1055     BC/PC
1056      BCAB
1057      BCAL
         ...  
11688     ZYME
11689     ZYNE
11690     ZYXI
11691      ZZK
11692      ZZZ
Name: symbol, Length: 6687, dtype: object

In [1]:
from alpha_vantage.alphaintelligence import AlphaIntelligence
from alpha_vantage.timeseries import TimeSeries

# data_tickers = pd.read_csv("datas//active_symbol.csv")
# data_tickers = pd.read_csv("datas//active_symbol_2.csv")
ai = AlphaIntelligence(key=API_KEY)
tickers = data_tickers.loc[(data_tickers['assetType'] == 'Stock') & (data_tickers['symbol'] >= 'BC/PA'), 'symbol'].to_list()
# data_tickers['sentiment_data'] = 'NA'

for itick in tickers:
    print(itick)
    try:
        # print(f"{itick}: ts")
        ts = TimeSeries(key=API_KEY, output_format='pandas')
        data, meta_data = ts.get_daily(symbol=itick, outputsize='full')
        data.to_parquet(f"datas//daily_ticks//{itick}_daily.parquet")
        data_tickers.loc[data_tickers['symbol']==itick, 'sentiment_data'] = "EXISTED"
    except:
        # print(f"{itick} dont have ts")
        data_tickers.loc[data_tickers['symbol']==itick, 'sentiment_data'] = "NA"
        
    try:
        # print(f"{itick}: sentiment")
        data, metadata = ai.get_news_sentiment(time_from='19750101T0001', limit=1000, tickers=itick)
        data['time_published'] = pd.to_datetime(data['time_published'])
        data_tickers.loc[data_tickers['symbol']==itick, 'sentiment_data'] = "EXISTED"
        data.to_parquet(f"datas//sentiments//{itick}_sentiment.parquet")
    except:
        # print(f"{itick} dont have sentiment")
        data_tickers.loc[data_tickers['symbol']==itick, 'sentiment_data'] = "NA"    

NameError: name 'API_KEY' is not defined

In [44]:
data_tickers.head()

,Unnamed: 0.1,Unnamed: 0,symbol,name,exchange,assetType,ipoDate,delistingDate,status,sentiment_data
0,0,0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,NaN,Active,EXISTED
1,1,1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,NaN,Active,EXISTED
2,2,2,AAA,ALTERNATIVE ACCESS FIRST PRIORITY CLO BOND ETF,NYSE ARCA,ETF,2020-09-09,NaN,Active,NaN
3,3,3,AAAU,Goldman Sachs Physical Gold ETF,BATS,ETF,2018-08-15,NaN,Active,NaN
4,4,4,AACG,ATA Creativity Global,NASDAQ,Stock,2008-01-29,NaN,Active,EXISTED


In [32]:
data_tickers = pd.read_csv("datas//active_symbol_2.csv")

In [37]:
data_tickers.loc[data_tickers['symbol']>='BC/PA', 'symbol'].index


Index([ 1053,  1054,  1055,  1056,  1057,  1058,  1059,  1060,  1061,  1062,
       ...
       11683, 11684, 11685, 11686, 11687, 11688, 11689, 11690, 11691, 11692],
      dtype='int64', length=10639)

# Sentiment data merge

In [4]:
from os import listdir
from os.path import isfile, join
import pandas as pd

mypath = 'datas\\sentiments'

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
tickers = [i.split("_")[0] for i in onlyfiles]

In [5]:
for icol in [onlyfiles[0]]:
    df_temp = pd.read_parquet(mypath + "\\" + icol)
    # df_temp['ticker'] = icol.split('_')[0]
    df_temp['date_published'] = pd.to_datetime(df_temp['time_published'])
    df_temp['date_published'] = df_temp['date_published'].dt.date
    display(df_temp.head())

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,date_published
0,ATA Creativity Global Schedules 2024 Second Qu...,https://www.globenewswire.com/news-release/202...,2024-08-01 12:30:00,[ATA Creativity Global],"BEIJING, Aug. 01, 2024 ( GLOBE NEWSWIRE ) -- A...",https://ml.globenewswire.com/Resource/Download...,GlobeNewswire,n/a,www.globenewswire.com,"[{'relevance_score': '0.360215', 'topic': 'Fin...",0.295523,Somewhat-Bullish,"[{'relevance_score': '0.522333', 'ticker': 'AA...",2024-08-01
1,ATA Creativity Global ( NASDAQ:AACG ) Covera...,https://www.defenseworld.net/2024/07/13/ata-cr...,2024-07-13 05:28:43,[Defense World Staff],Research analysts at StockNews.com began cover...,https://www.marketbeat.com/logos/ata-creativit...,Stocknews.com,n/a,www.defenseworld.net,"[{'relevance_score': '0.744043', 'topic': 'Ear...",0.105572,Neutral,"[{'relevance_score': '0.817578', 'ticker': 'AA...",2024-07-13
2,ATA Creativity Global ( NASDAQ:AACG ) Resear...,https://www.defenseworld.net/2024/07/05/ata-cr...,2024-07-05 05:28:44,[Defense World Staff],StockNews.com assumed coverage on shares of AT...,https://www.marketbeat.com/logos/ata-creativit...,Stocknews.com,n/a,www.defenseworld.net,"[{'relevance_score': '0.917436', 'topic': 'Ear...",0.115629,Neutral,"[{'relevance_score': '0.877135', 'ticker': 'AA...",2024-07-05
3,ATA Creativity Global ( NASDAQ:AACG ) Earns ...,https://www.defenseworld.net/2024/06/27/ata-cr...,2024-06-27 10:11:07,[Defense World Staff],Equities research analysts at StockNews.com be...,https://www.marketbeat.com/logos/ata-creativit...,Stocknews.com,n/a,www.defenseworld.net,"[{'relevance_score': '0.818451', 'topic': 'Ear...",0.112501,Neutral,"[{'relevance_score': '0.853391', 'ticker': 'AA...",2024-06-27
4,ATA Creativity Global ( NASDAQ:AACG ) Resear...,https://www.defenseworld.net/2024/06/19/ata-cr...,2024-06-19 05:18:53,[Defense World Staff],Research analysts at StockNews.com initiated c...,https://www.marketbeat.com/logos/ata-creativit...,Stocknews.com,n/a,www.defenseworld.net,"[{'relevance_score': '0.818451', 'topic': 'Ear...",0.122886,Neutral,"[{'relevance_score': '0.798255', 'ticker': 'AA...",2024-06-19


In [3]:
from os import listdir
from os.path import isfile, join
import datetime as dt

mypath = 'datas//sentiments'

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
tickers = [i.split("_")[0] for i in onlyfiles]

col_base_use = ['date_published', 'title', 'url', 'time_published', 'authors', 'summary',
       'source', 'overall_sentiment_score', 'overall_sentiment_label']

tickers = pd.Series(tickers, dtype='object')
tickers = tickers[tickers>='LEA'].to_list()

df_main = pd.DataFrame()

for icol in tickers:
    print(icol)
    try:
        df_temp = pd.read_parquet( "datas\\sentiments\\" + icol + "_sentiment.parquet")
        # df_temp['ticker'] = icol.split('_')[0]
        df_temp['date_published'] = pd.to_datetime(df_temp['time_published'])
        df_temp['date_published'] = pd.to_datetime(df_temp['date_published'].dt.date)
        
        df_label_main = pd.read_parquet(f'datas//daily_ticks//{icol}_daily.parquet')
        data = df_label_main.copy()
    
        td2 = dt.timedelta(days=2)
        td3 = dt.timedelta(days=3)
        td7 = dt.timedelta(days=5)
        td14 = dt.timedelta(days=10)
        td21 = dt.timedelta(days=15)
        td30 = dt.timedelta(days=20)
        
        for idate in data.index:
            df_label_main.loc[idate, 'label_close2max'] = data.loc[(data.index >= idate) & (data.index <= idate + td2), '4. close'].max()
            df_label_main.loc[idate, 'label_close3max'] = data.loc[(data.index >= idate) & (data.index <= idate + td3), '4. close'].max()
            df_label_main.loc[idate, 'label_close5max'] = data.loc[(data.index >= idate) & (data.index <= idate + td7), '4. close'].max()
            df_label_main.loc[idate, 'label_close10max'] = data.loc[(data.index >= idate) & (data.index <= idate + td14), '4. close'].max()
            df_label_main.loc[idate, 'label_close15max'] = data.loc[(data.index >= idate) & (data.index <= idate + td21), '4. close'].max()
            df_label_main.loc[idate, 'label_close20max'] = data.loc[(data.index >= idate) & (data.index <= idate + td30), '4. close'].max()
    
            df_label_main.loc[idate, 'label_close2min'] = data.loc[(data.index >= idate) & (data.index <= idate + td2), '4. close'].min()
            df_label_main.loc[idate, 'label_close3min'] = data.loc[(data.index >= idate) & (data.index <= idate + td3), '4. close'].min()
            df_label_main.loc[idate, 'label_close5min'] = data.loc[(data.index >= idate) & (data.index <= idate + td7), '4. close'].min()
            df_label_main.loc[idate, 'label_close10min'] = data.loc[(data.index >= idate) & (data.index <= idate + td14), '4. close'].min()
            df_label_main.loc[idate, 'label_close15min'] = data.loc[(data.index >= idate) & (data.index <= idate + td21), '4. close'].min()
            df_label_main.loc[idate, 'label_close20min'] = data.loc[(data.index >= idate) & (data.index <= idate + td30), '4. close'].min()
    
            df_label_main.loc[idate, 'label_close2mean'] = data.loc[(data.index >= idate) & (data.index <= idate + td2), '4. close'].mean()
            df_label_main.loc[idate, 'label_close3mean'] = data.loc[(data.index >= idate) & (data.index <= idate + td3), '4. close'].mean()
            df_label_main.loc[idate, 'label_close5mean'] = data.loc[(data.index >= idate) & (data.index <= idate + td7), '4. close'].mean()
            df_label_main.loc[idate, 'label_close10mean'] = data.loc[(data.index >= idate) & (data.index <= idate + td14), '4. close'].mean()
            df_label_main.loc[idate, 'label_close15mean'] = data.loc[(data.index >= idate) & (data.index <= idate + td21), '4. close'].mean()
            df_label_main.loc[idate, 'label_close20mean'] = data.loc[(data.index >= idate) & (data.index <= idate + td30), '4. close'].mean()
    
        df_label_main['label_close2max'] = (df_label_main.label_close2max - data['4. close'])/ df_label_main['4. close']
        df_label_main['label_close3max'] = (df_label_main.label_close3max - data['4. close'])/ df_label_main['4. close']
        df_label_main['label_close5max'] = (df_label_main.label_close5max - data['4. close'])/ df_label_main['4. close']
        df_label_main['label_close10max'] = (df_label_main.label_close10max - data['4. close'])/ df_label_main['4. close']
        df_label_main['label_close15max'] = (df_label_main.label_close15max - data['4. close'])/ df_label_main['4. close']
        df_label_main['label_close20max'] = (df_label_main.label_close20max - data['4. close'])/ df_label_main['4. close']
        
        df_label_main['label_close2min'] = (df_label_main.label_close2min - data['4. close'])/ df_label_main['4. close']
        df_label_main['label_close3min'] = (df_label_main.label_close3min - data['4. close'])/ df_label_main['4. close']
        df_label_main['label_close5min'] = (df_label_main.label_close5min - data['4. close'])/ df_label_main['4. close']
        df_label_main['label_close10min'] = (df_label_main.label_close10min - data['4. close'])/ df_label_main['4. close']
        df_label_main['label_close15min'] = (df_label_main.label_close15min - data['4. close'])/ df_label_main['4. close']
        df_label_main['label_close20min'] = (df_label_main.label_close20min - data['4. close'])/ df_label_main['4. close']
    
        df_label_main['label_close2mean'] = (df_label_main.label_close2mean - data['4. close'])/ df_label_main['4. close']
        df_label_main['label_close3mean'] = (df_label_main.label_close3mean - data['4. close'])/ df_label_main['4. close']
        df_label_main['label_close5mean'] = (df_label_main.label_close5mean - data['4. close'])/ df_label_main['4. close']
        df_label_main['label_close10mean'] = (df_label_main.label_close10mean - data['4. close'])/ df_label_main['4. close']
        df_label_main['label_close15mean'] = (df_label_main.label_close15mean - data['4. close'])/ df_label_main['4. close']
        df_label_main['label_close20mean'] = (df_label_main.label_close20mean - data['4. close'])/ df_label_main['4. close']
        
        ## Increasing prie buy
        df_label_main['inc_close2mean_perc5'] = 0
        df_label_main.loc[df_label_main["label_close2mean"] >= 0.05, 'inc_close2mean_perc5'] = 1
        df_label_main['inc_close2mean_perc10'] = 0
        df_label_main.loc[df_label_main["label_close2mean"] >= 0.10, 'inc_close2mean_perc10'] = 1
        df_label_main['inc_close2mean_perc3'] = 0
        df_label_main.loc[df_label_main["label_close2mean"] >= 0.03, 'inc_close2mean_perc3'] = 1
        
        df_label_main['inc_close2max_perc5'] = 0
        df_label_main.loc[df_label_main["label_close2max"] >= 0.05, 'inc_close2max_perc5'] = 1
        df_label_main['inc_close2max_perc10'] = 0
        df_label_main.loc[df_label_main["label_close2max"] >= 0.10, 'inc_close2max_perc10'] = 1
        df_label_main['inc_close2max_perc3'] = 0
        df_label_main.loc[df_label_main["label_close2max"] >= 0.03, 'inc_close2max_perc3'] = 1
        
        df_label_main['inc_close3mean_perc5'] = 0
        df_label_main.loc[df_label_main["label_close3mean"] >= 0.05, 'inc_close3mean_perc5'] = 1
        df_label_main['inc_close3mean_perc10'] = 0
        df_label_main.loc[df_label_main["label_close3mean"] >= 0.10, 'inc_close3mean_perc10'] = 1
        df_label_main['inc_close3mean_perc3'] = 0
        df_label_main.loc[df_label_main["label_close3mean"] >= 0.03, 'inc_close3mean_perc3'] = 1
        
        df_label_main['inc_close3max_perc5'] = 0
        df_label_main.loc[df_label_main["label_close3max"] >= 0.05, 'inc_close3max_perc5'] = 1
        df_label_main['inc_close3max_perc10'] = 0
        df_label_main.loc[df_label_main["label_close3max"] >= 0.10, 'inc_close3max_perc10'] = 1
        df_label_main['inc_close3max_perc3'] = 0
        df_label_main.loc[df_label_main["label_close3max"] >= 0.03, 'inc_close3max_perc3'] = 1
        
        df_label_main['inc_close5mean_perc5'] = 0
        df_label_main.loc[df_label_main["label_close5mean"] >= 0.05, 'inc_close5mean_perc5'] = 1
        df_label_main['inc_close5mean_perc10'] = 0
        df_label_main.loc[df_label_main["label_close5mean"] >= 0.10, 'inc_close5mean_perc10'] = 1
        df_label_main['inc_close5mean_perc3'] = 0
        df_label_main.loc[df_label_main["label_close5mean"] >= 0.03, 'inc_close5mean_perc3'] = 1
        
        df_label_main['inc_close5max_perc5'] = 0
        df_label_main.loc[df_label_main["label_close5max"] >= 0.05, 'inc_close5max_perc5'] = 1
        df_label_main['inc_close5max_perc10'] = 0
        df_label_main.loc[df_label_main["label_close5max"] >= 0.10, 'inc_close5max_perc10'] = 1
        df_label_main['inc_close5max_perc3'] = 0
        df_label_main.loc[df_label_main["label_close5max"] >= 0.03, 'inc_close5max_perc3'] = 1
        
        df_label_main['inc_close10mean_perc5'] = 0
        df_label_main.loc[df_label_main["label_close10mean"] >= 0.05, 'inc_close10mean_perc5'] = 1
        df_label_main['inc_close10mean_perc10'] = 0
        df_label_main.loc[df_label_main["label_close10mean"] >= 0.10, 'inc_close10mean_perc10'] = 1
        df_label_main['inc_close10mean_perc3'] = 0
        df_label_main.loc[df_label_main["label_close10mean"] >= 0.03, 'inc_close10mean_perc3'] = 1
        
        df_label_main['inc_close10max_perc5'] = 0
        df_label_main.loc[df_label_main["label_close10max"] >= 0.05, 'inc_close10max_perc5'] = 1
        df_label_main['inc_close10max_perc10'] = 0
        df_label_main.loc[df_label_main["label_close10max"] >= 0.10, 'inc_close10max_perc10'] = 1
        df_label_main['inc_close10max_perc3'] = 0
        df_label_main.loc[df_label_main["label_close10max"] >= 0.03, 'inc_close10max_perc3'] = 1
        
        ## Decreasing price --sell
        df_label_main['dec_close2mean_perc5'] = 0
        df_label_main.loc[df_label_main["label_close2mean"] <= -0.05, 'dec_close2mean_perc5'] = 1
        df_label_main['dec_close2mean_perc10'] = 0
        df_label_main.loc[df_label_main["label_close2mean"] <= -0.10, 'dec_close2mean_perc10'] = 1
        df_label_main['dec_close2mean_perc3'] = 0
        df_label_main.loc[df_label_main["label_close2mean"] <= -0.03, 'dec_close2mean_perc3'] = 1
        
        df_label_main['dec_close2min_perc5'] = 0
        df_label_main.loc[df_label_main["label_close2min"] <= -0.05, 'dec_close2min_perc5'] = 1
        df_label_main['dec_close2min_perc10'] = 0
        df_label_main.loc[df_label_main["label_close2min"] <= -0.10, 'dec_close2min_perc10'] = 1
        df_label_main['dec_close2min_perc3'] = 0
        df_label_main.loc[df_label_main["label_close2min"] <= -0.03, 'dec_close2min_perc3'] = 1
    
        df_label_main['dec_close3mean_perc5'] = 0
        df_label_main.loc[df_label_main["label_close3mean"] <= -0.05, 'dec_close3mean_perc5'] = 1
        df_label_main['dec_close3mean_perc10'] = 0
        df_label_main.loc[df_label_main["label_close3mean"] <= -0.10, 'dec_close3mean_perc10'] = 1
        df_label_main['dec_close3mean_perc3'] = 0
        df_label_main.loc[df_label_main["label_close3mean"] <= -0.03, 'dec_close3mean_perc3'] = 1
        
        df_label_main['dec_close3min_perc5'] = 0
        df_label_main.loc[df_label_main["label_close3min"] <= -0.05, 'dec_close3min_perc5'] = 1
        df_label_main['dec_close3min_perc10'] = 0
        df_label_main.loc[df_label_main["label_close3min"] <= -0.10, 'dec_close3min_perc10'] = 1
        df_label_main['dec_close3min_perc3'] = 0
        df_label_main.loc[df_label_main["label_close3min"] <= -0.03, 'dec_close3min_perc3'] = 1
        
        df_label_main['dec_close5mean_perc5'] = 0
        df_label_main.loc[df_label_main["label_close5mean"] <= -0.05, 'dec_close5mean_perc5'] = 1
        df_label_main['dec_close5mean_perc10'] = 0
        df_label_main.loc[df_label_main["label_close5mean"] <= -0.10, 'dec_close5mean_perc10'] = 1
        df_label_main['dec_close5mean_perc3'] = 0
        df_label_main.loc[df_label_main["label_close5mean"] <= -0.03, 'dec_close5mean_perc3'] = 1
        
        df_label_main['dec_close5min_perc5'] = 0
        df_label_main.loc[df_label_main["label_close5min"] <= -0.05, 'dec_close5min_perc5'] = 1
        df_label_main['dec_close5min_perc10'] = 0
        df_label_main.loc[df_label_main["label_close5min"] <= -0.10, 'dec_close5min_perc10'] = 1
        df_label_main['dec_close5min_perc3'] = 0
        df_label_main.loc[df_label_main["label_close5min"] <= -0.03, 'dec_close5min_perc3'] = 1
        
        df_label_main['dec_close10mean_perc5'] = 0
        df_label_main.loc[df_label_main["label_close10mean"] <= -0.05, 'dec_close10mean_perc5'] = 1
        df_label_main['dec_close10mean_perc10'] = 0
        df_label_main.loc[df_label_main["label_close10mean"] <= -0.10, 'dec_close10mean_perc10'] = 1
        df_label_main['dec_close10mean_perc3'] = 0
        df_label_main.loc[df_label_main["label_close10mean"] <= -0.03, 'dec_close10mean_perc3'] = 1
        
        df_label_main['dec_close10min_perc5'] = 0
        df_label_main.loc[df_label_main["label_close10min"] <= -0.05, 'dec_close10min_perc5'] = 1
        df_label_main['dec_close10min_perc10'] = 0
        df_label_main.loc[df_label_main["label_close10min"] <= -0.10, 'dec_close10min_perc10'] = 1
        df_label_main['dec_close10min_perc3'] = 0
        df_label_main.loc[df_label_main["label_close10min"] <= -0.03, 'dec_close10min_perc3'] = 1
    
        df_label_main['date_published'] = df_label_main.index.values
        df_sentiments = df_temp[col_base_use].merge(df_label_main, on='date_published', how='left')
    
        if len(df_main) == 0:
            df_main = df_sentiments.copy()
        else:
            df_main = pd.concat([df_main, df_sentiments], axis=0, ignore_index=True)
    
        df_sentiments.to_parquet(f"datas//sentiment_labels//{icol}_sentiment_label.parquet")
    except:
        print("skip")
        continue

LEA
LECO
LEDS
LEE
LEGH
LEGN
LEG
LENZ
LEN
LESL
LEV
LEXX
LFCR
LFLY
LFMD
LFST
LFT
LFUS
LFVN
LFWD
LGCB
LGCL
LGF-A
LGHL
LGIH
LGMK
LGND
LGO
LGVN
LHX
LH
LICN
LICY
LIDR
LIFW
LIF
LII
LILA
LILM
LINC
LIND
LINE
LINK
LIN
LIONU
LION
LIPO
LIQT
LITB
LITE
LITM
LITT
LIVE
LIVN
LIXT
LI
LKCO
LKFN
LKQ
LLAP
LLY
LMACU
LMAT
LMB
LMFA
LMND
LMNL
LMNR
LMT
LNC
LNKB
LNN
LNSR
LNTH
LNT
LNW
LNZA
LOAN
LOAR
LOBO
LOB
LOCL
LOCO
LOGC
LOGI
LOOP
LOPE
LOT
LOVE
LPAAU
LPCN
LPG
LPLA
LPL
LPRO
LPSN
LPTH
LPTX
LPX
LQDA
LQDT
LQR
LRCX
LRE
LRFC
LRHC
LRMR
LRN
LSAK
LSBK
LSB
LSCC
LSEA
LSH
LSPD
LSTA
LSTR
LTBR
LTC
LTH
LTM
LTRN
LTRX
LTRY
LUCD
LUCY
LULU
LUMN
LUMO
LUNA
LUNG
LUNR
LUV
LUXH
LU
LVAC
LVLU
LVO
LVRO
LVS
LVTX
LVWR
LWAY
LWLG
LW
LXEH
LXEO
LXFR
LXP
LXRX
LXU
LX
LYB
LYEL
LYG
LYRA
LYTS
LYT
LYV
LZB
LZM
LZ
MAA
MACK
MAC
MAIN
MAMA
MAMO
MANH
MANU
MAN
MAPS
MARA
MARPS
MARX
MAR
MASI
MASS
MAS
MATH
MATV
MATW
MATX
MAT
MAXN
MAX
MAYS
MA
MBAVU
MBCN
MBC
MBIN
MBIO
MBI
MBLY
MBNKP
MBOT
MBRX
MBUU
MBWM
MBX
MCAA
MCACR
MCBS
MCB
MCD
MCFT
MCHP
MCHX
MCK
MCO
MCRB
MCR

In [ ]:
df_main.to_parquet("datas\\sentiment_labels\\all_stocks.parquet")